In [78]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

from tqdm import tqdm
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import leaguegamelog

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, ElasticNet, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

pd.options.display.max_columns=200

In [18]:
seasons = ["20{}-{}".format(x, x+1) for x in range(10, 21)]
print(seasons)
def gather_data(seasons = seasons):
    full_player_data = []
    for season in seasons:
        player_gls = pd.read_csv('./data/player_gamelogs_{}.csv'.format(season), dtype={'GAME_ID':'int'})
        player_gls_adv = pd.read_csv('./data/player_advanced_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'int'})
        player_gls_scoring = pd.read_csv('./data/player_scoring_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'int'})
#         player_gls_tracking = pd.read_csv('./data/player_tracking_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'object'})
    
        print("player_gls_shape:", player_gls.shape,
             "player_gls_adv_shape:", player_gls_adv.shape,
             "player_gls_scoring_shape:", player_gls_scoring.shape)
    
        player_gls = player_gls[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']]

        player_full = pd.merge(player_gls, player_gls_adv, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
        player_full = pd.merge(player_full, player_gls_scoring, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
    
        print("after_merging shape", player_full.shape)
        
        # drop unnecessary columns
        player_full = player_full.drop(columns = ['TEAM_ID', 'TEAM_NAME', 'TEAM_ID_x',
                         'TEAM_ABBREVIATION_x','TEAM_CITY','PLAYER_NAME_x','START_POSITION',
                          'COMMENT','MIN_x','TEAM_ID_x', 'TEAM_ABBREVIATION_x','TEAM_CITY_x', 
                          'PLAYER_NAME_x', 'START_POSITION_x', 'COMMENT_x', 'MIN_x', 
                         'TEAM_ID_x', 'TEAM_ABBREVIATION_x', 'TEAM_CITY_x', 'PLAYER_NAME_x', 
                          'START_POSITION_x', 'COMMENT_x', 'MIN_x', 'AST_x', 'FG_PCT_x',
                         'FG_PCT', 'FG3_PCT', 'FT_PCT', 'E_NET_RATING', 'NET_RATING', 
                         'AST_PCT', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'EFG_PCT', 'TS_PCT', 
                         'PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_FT', 'CFG_PCT',
                        'UFG_PCT', 'DFG_PCT'], 
                       errors= 'ignore')

        # Convert date to datetime
        player_full['GAME_DATE'] = pd.to_datetime(player_full['GAME_DATE'])
        
        # Convert W/L to a binary 1/0 for win/loss
        player_full['WL'] = (player_full['WL'].str[0] == 'W').astype(int)
        
        # Add a binary home_game feature
        player_full['HOME_GAME'] = 0 
        player_full['HOME_GAME'] = (player_full['MATCHUP'].str[:6] == player_full['TEAM_ABBREVIATION'].str[:] + ' vs').astype(int)
        
        # convert the percentages into raw numbers (because we will be averaging them, we don't want to average percentages)
        # for example, percentage of points from midrange will be points scored from midrange
        player_full['FG2M'] = player_full['FGM'] - player_full['FG3M']
        player_full['FG2A'] = player_full['FGA'] - player_full['FG3A']
        player_full['PTS_2PT'] = (player_full['PTS'] * player_full['PCT_PTS_2PT']).astype('int8')
        player_full['PTS_2PT_MR'] = (player_full['PTS'] * player_full['PCT_PTS_2PT_MR']).astype('int8')
        player_full['PTS_3PT'] = (player_full['PTS'] * player_full['PCT_PTS_3PT']).astype('int8')
        player_full['PTS_FB'] = (player_full['PTS'] * player_full['PCT_PTS_FB']).astype('int8')
        player_full['PTS_OFF_TOV'] = (player_full['PTS'] * player_full['PCT_PTS_OFF_TOV']).astype('int8')
        player_full['PTS_PAINT'] = (player_full['PTS'] * player_full['PCT_PTS_PAINT']).astype('int8')
        player_full['AST_2PM'] = (player_full['FG2M'] * player_full['PCT_AST_2PM']).astype('int8')
        player_full['UAST_2PM'] = (player_full['FG2M'] * player_full['PCT_UAST_2PM']).astype('int8')
        player_full['AST_3PM'] = (player_full['FG3M'] * player_full['PCT_AST_3PM']).astype('int8')
        player_full['UAST_3PM'] = (player_full['FG3M'] * player_full['PCT_UAST_3PM']).astype('int8')



        player_full = player_full.drop(columns = ['PCT_PTS_2PT', 'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 
                                                  'PCT_PTS_FB', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT',
                                                  'PCT_AST_2PM', 'PCT_UAST_2PM', 'PCT_AST_3PM',
                                                  'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM', 
                                                  'FGM', 'FGA', 'AST_TOV', 'USG_PCT', 'PACE',
                                                  'PACE_PER40', ], errors='ignore')
        
        
        full_player_data.append(player_full)
        
        
    
    player_df = pd.concat(full_player_data)
    
    
    player_df['home_team_abbr'] = player_df.apply(lambda row: row['TEAM_ABBREVIATION'] if row['HOME_GAME'] == 1 else row['MATCHUP'][-3:], axis=1)
    player_df['away_team_abbr'] = player_df.apply(lambda row: row['TEAM_ABBREVIATION'] if row['HOME_GAME'] == 0 else row['MATCHUP'][-3:], axis=1)
    
    player_df[['home_team_abbr', 'away_team_abbr']] = player_df[['home_team_abbr', 'away_team_abbr']].replace({'NOH':'NOP',
                                                                                                               'NJN':'BKN'})
    
    # Reorder columns
    player_df = player_df[['SEASON_YEAR', 'TEAM_ABBREVIATION', 'PLAYER_NAME', 'PLAYER_ID', 
                           'home_team_abbr', 'away_team_abbr', 'GAME_ID', 
                           'GAME_DATE', 'MATCHUP', 'WL', 'HOME_GAME', 'MIN',
                           'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
                           'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD',
                           'PTS', 'PLUS_MINUS', 'E_OFF_RATING', 'E_DEF_RATING',
                           'AST_RATIO', 'TM_TOV_PCT', 'E_USG_PCT',
                           'E_PACE', 'POSS', 'PIE',
                           'FG2M', 'FG2A', 'PTS_2PT_MR', 'PTS_FB', 
                           'PTS_OFF_TOV', 'PTS_PAINT', 'AST_2PM',
                           'UAST_2PM', 'AST_3PM', 'UAST_3PM']]
    
    return player_df

player_df = gather_data(seasons)

['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21']
player_gls_shape: (25153, 64) player_gls_adv_shape: (31419, 31) player_gls_scoring_shape: (29475, 24)
after_merging shape (25153, 83)
player_gls_shape: (20758, 64) player_gls_adv_shape: (27638, 31) player_gls_scoring_shape: (25455, 24)
after_merging shape (20758, 83)
player_gls_shape: (25757, 64) player_gls_adv_shape: (33542, 31) player_gls_scoring_shape: (31338, 24)
after_merging shape (25757, 83)
player_gls_shape: (25618, 64) player_gls_adv_shape: (33681, 31) player_gls_scoring_shape: (31378, 24)
after_merging shape (25618, 83)
player_gls_shape: (25981, 64) player_gls_adv_shape: (33522, 31) player_gls_scoring_shape: (31412, 24)
after_merging shape (25981, 83)
player_gls_shape: (26078, 64) player_gls_adv_shape: (33659, 31) player_gls_scoring_shape: (31423, 24)
after_merging shape (26078, 83)
player_gls_shape: (26139, 64) player_gls_adv_shape: (33610, 31) player_gls

In [88]:
player_df

,SEASON_YEAR,TEAM_ABBREVIATION,PLAYER_NAME,PLAYER_ID,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,MIN,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,E_DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,E_PACE,POSS,PIE,FG2M,FG2A,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
0,2010-11,MIL,Corey Maggette,1894,OKC,MIL,21001225,2011-04-13,MIL @ OKC,1,0,18.776667,0,0,6,6,0,1,2,2,1,0,0,2,3,12,2,111.0,105.4,16.7,16.7,0.239,96.89,39,0.139,3,5,3,2,0,2,3,0,0,0
1,2010-11,NOH,Jarrett Jack,101127,DAL,NOP,21001223,2011-04-13,NOH @ DAL,0,0,28.658333,1,1,5,6,0,0,7,1,0,1,0,2,4,22,-22,100.3,137.0,28.0,4.0,0.296,95.91,56,0.144,7,14,6,2,0,8,2,4,1,0
2,2010-11,NJN,Jordan Farmar,200770,CHI,BKN,21001222,2011-04-13,NJN @ CHI,0,0,38.166667,2,6,5,6,0,4,12,4,0,2,0,1,4,21,7,107.9,102.8,35.3,11.8,0.274,91.28,71,0.198,5,9,3,1,6,6,0,5,1,1
3,2010-11,NYK,Amar'e Stoudemire,2405,BOS,NYK,21001216,2011-04-13,NYK @ BOS,0,0,20.450000,0,0,2,3,2,2,1,2,0,1,0,1,4,14,-3,107.6,107.5,5.3,10.5,0.384,101.49,40,0.072,6,15,4,0,0,7,1,4,0,0
4,2010-11,OKC,Byron Mullens,201957,OKC,MIL,21001225,2011-04-13,OKC vs. MIL,0,1,29.178333,0,0,2,4,1,4,0,2,1,0,1,2,2,10,-22,90.1,130.8,0.0,15.4,0.191,97.12,60,0.041,4,9,0,1,4,8,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8689,2020-21,GSW,Jordan Poole,1629673,BKN,GSW,22000001,2020-12-22,GSW @ BKN,0,0,17.916667,0,1,0,0,0,2,3,2,0,0,0,2,1,2,-3,83.8,94.8,33.3,22.2,0.135,103.63,38,0.000,1,3,0,0,0,2,0,1,0,0
8690,2020-21,GSW,Andrew Wiggins,203952,BKN,GSW,22000001,2020-12-22,GSW @ BKN,0,0,31.233333,2,6,3,4,0,2,1,4,0,1,2,4,4,13,-28,74.4,115.8,4.3,17.4,0.262,111.88,72,-0.044,2,10,2,0,0,2,1,1,2,0
8691,2020-21,GSW,Juan Toscano-Anderson,1629308,BKN,GSW,22000001,2020-12-22,GSW @ BKN,0,0,13.250000,0,0,0,0,0,4,2,0,0,2,0,3,1,4,-9,70.5,95.5,50.0,0.0,0.058,119.84,33,0.208,2,2,0,2,0,4,2,0,0,0
8692,2020-21,LAC,Luke Kennard,1628379,LAL,LAC,22000002,2020-12-22,LAC @ LAL,1,0,21.055000,0,3,0,0,0,2,2,1,2,1,0,3,0,4,-3,116.9,112.2,22.2,11.1,0.144,104.59,44,0.032,2,3,0,0,0,4,1,1,0,0


In [26]:
def aggregate_player_stats(df = player_df):

    seasons = player_df['SEASON_YEAR'].unique()

    df = df.sort_values('GAME_DATE')

    for season in seasons:
        season_df = df.loc[df['SEASON_YEAR'] == season]
        player_ids = season_df['PLAYER_ID'].unique()
        for p_id in tqdm(player_ids, desc='Progress'):
            player_log = season_df.loc[season_df['PLAYER_ID'] == p_id]

            avg_player_data = player_log.iloc[:, 11:].shift(1).rolling(10, min_periods=5).mean()

            cols = avg_player_data.columns

            df.loc[(df['SEASON_YEAR'] == season) & (df['PLAYER_ID'] == p_id), cols] = avg_player_data

    df.to_csv('player_avg_last10.csv', index=False)
        
    return df
    
player_stats = aggregate_player_stats(df=player_df)

Progress: 100%|██████████████████████████████████████████████████████████████████████| 480/480 [00:44<00:00, 10.85it/s]


In [50]:
# Drop Null Rows
player_stats = player_stats.dropna(axis=0)

In [82]:
player_stats['GAME_ID'].nunique()

11515

In [117]:
def prep_data_for_model(df = player_stats):
    matchup_info = []
    matchups = []
    targets = []
    
    all_team_gamelogs = pd.read_csv('./data/team_gamelogs_2000_present.csv')

    game_results = all_team_gamelogs.loc[all_team_gamelogs['MATCHUP'].str.contains('vs'), ['GAME_ID', 'PLUS_MINUS']]
    game_results

    for game_id in tqdm(df['GAME_ID'].unique(), desc="progress"):
        game_df = df.loc[df['GAME_ID'] == game_id]
        
        point_diff = game_results.loc[game_results['GAME_ID'] == game_id, 'PLUS_MINUS'].values[0]
        targets.append(point_diff)

        home = test.loc[test['HOME_GAME'] == 1].sort_values('MIN', ascending=False)
        home_stats = home.iloc[:, 11:].values

        if home_stats.shape[0] < 10:
            missing_rows = 10 - home_stats.shape[0]
            padding = np.zeros((missing_rows, home_stats.shape[1]))
            home_stats = np.concatenate([home_stats, padding], axis=0)

        away = test.loc[test['HOME_GAME'] == 0].sort_values('MIN', ascending=False)
        away_stats = away.iloc[:, 11:].values

        if away_stats.shape[0] < 10:
            missing_rows = 10 - away_stats.shape[0]
            padding = np.zeros((missing_rows, away_stats.shape[1]))
            away_stats = np.concatenate([away_stats, padding], axis=0)


        matchup = (np.concatenate([home_stats, away_stats], axis=1), point_diff)
        matchups.append(matchup)

    matchup_features = np.stack(matchups, axis=0)
    return matchup_features, np.array(targets)

matchup_features, targets = prep_data_for_model(df = player_stats)

progress: 100%|█████████████████████████████████████████████████████████████████| 11515/11515 [00:43<00:00, 263.77it/s]


In [118]:
matchup_data.shape, targets.shape

((11515, 10, 68), (11515,))

In [136]:
# Split up data - training set and testing set
X_train, X_test, y_train, y_test = train_test_split(matchup_data, targets, test_size=0.2)

X_train.shape, X_test.shape

((9212, 10, 68), (2303, 10, 68))

In [198]:
torch.tensor([0,1,2,4]).float()

tensor([0., 1., 2., 4.])

In [205]:
class PlayerBoxScoreMatchupsDataset(Dataset):
    """"""
    def __init__(self, matchup_data, targets):
        self.targets = targets
        self.matchup_data = matchup_data
        
    def __len__(self):
        return len(self.matchup_data)
    
    def __getitem__(self, index):
        X = matchup_data[index]
        y = targets[index]
        
        return X, y
    

In [206]:
training_set = PlayerBoxScoreMatchupsDataset(X_train, y_train)
validation_set = PlayerBoxScoreMatchupsDataset(X_test, y_test)

train_loader = DataLoader(training_set, batch_size=100, shuffle=False)
val_loader = DataLoader(training_set, batch_size=100, shuffle=False)
for i, batch in enumerate(loader): 
    print("idx:", i)
    print("features:", batch[0])
    print("targets:", batch[1])
    break

idx: 0
features: tensor([[[35.4694,  1.0000,  3.1667,  ...,  1.4000,  0.0000,  0.0000],
         [32.9544,  1.0000,  3.0000,  ...,  0.8000,  0.8000,  0.0000],
         [32.0394,  0.1667,  0.3333,  ...,  4.2000,  1.0000,  0.2000],
         ...,
         [19.2931,  0.0000,  0.0000,  ...,  1.0000,  0.0000,  0.0000],
         [15.9469,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[35.4694,  1.0000,  3.1667,  ...,  1.4000,  0.0000,  0.0000],
         [32.9544,  1.0000,  3.0000,  ...,  0.8000,  0.8000,  0.0000],
         [32.0394,  0.1667,  0.3333,  ...,  4.2000,  1.0000,  0.2000],
         ...,
         [19.2931,  0.0000,  0.0000,  ...,  1.0000,  0.0000,  0.0000],
         [15.9469,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[35.4694,  1.0000,  3.1667,  ...,  1.4000,  0.0000,  0.0000],
         [32.9544,  1.0000, 

In [207]:
type(matchup_data[0])

numpy.ndarray

In [208]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv1d(10, 1, kernel_size=1, stride=1)
        
        self.fc1 = nn.Linear(68, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
        
        self.dropout = nn.Dropout(0.25)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
#         print(x.shape)
        x = x.view(-1, 68)
#         print(x.shape)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv1d(10, 1, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=68, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [203]:
len(val_loader.sampler)

9212

In [211]:
import torch.optim as optim

loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

epochs = 30

for e in range(epochs):
    train_loss = 0
    val_loss = 0
    net.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        
        output = net(data.float())
        
        loss = loss_function(output, target.float())
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        
    net.eval()
    for data, target in val_loader:
        output = net(data.float())
        loss = loss_function(output, target.float())
        
        val_loss += loss.item()*data.size(0)
        
    train_loss = train_loss / len(train_loader.sampler)
    
    val_loss = val_loss /len(val_loader.sampler)


    print('Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f}'.format(
    e, train_loss, val_loss))
    

Epoch: 0 	Training Loss: 178.8775 	Validation Loss: 178.8831
Epoch: 1 	Training Loss: 178.8937 	Validation Loss: 178.9535
Epoch: 2 	Training Loss: 178.8650 	Validation Loss: 179.1013
Epoch: 3 	Training Loss: 178.8599 	Validation Loss: 178.8092
Epoch: 4 	Training Loss: 178.8449 	Validation Loss: 178.9068
Epoch: 5 	Training Loss: 178.8046 	Validation Loss: 178.8974
Epoch: 6 	Training Loss: 178.8172 	Validation Loss: 178.7714
Epoch: 7 	Training Loss: 178.8067 	Validation Loss: 178.9023
Epoch: 8 	Training Loss: 178.7671 	Validation Loss: 178.9164
Epoch: 9 	Training Loss: 178.7558 	Validation Loss: 178.9866
Epoch: 10 	Training Loss: 178.7863 	Validation Loss: 178.9690
Epoch: 11 	Training Loss: 178.7706 	Validation Loss: 178.8247
Epoch: 12 	Training Loss: 178.7144 	Validation Loss: 178.9111
Epoch: 13 	Training Loss: 178.7362 	Validation Loss: 178.7384
Epoch: 14 	Training Loss: 178.7239 	Validation Loss: 179.1338
Epoch: 15 	Training Loss: 178.7732 	Validation Loss: 178.7312
Epoch: 16 	Trainin

In [188]:
net(test_input.type(torch.float))

torch.Size([100, 1, 68])
torch.Size([100, 68])


tensor([[-0.1018],
        [-0.0696],
        [-0.0693],
        [-0.1368],
        [-0.0764],
        [-0.0737],
        [-0.1385],
        [-0.0279],
        [-0.0863],
        [-0.0772],
        [-0.0931],
        [-0.0613],
        [-0.0631],
        [-0.0309],
        [-0.1466],
        [-0.0946],
        [-0.0900],
        [-0.0721],
        [-0.0578],
        [-0.1200],
        [-0.0390],
        [-0.0851],
        [ 0.0129],
        [-0.0701],
        [-0.1577],
        [-0.0531],
        [-0.0988],
        [-0.0422],
        [-0.1033],
        [-0.1155],
        [-0.0796],
        [-0.0670],
        [-0.1114],
        [-0.1087],
        [-0.0338],
        [-0.1232],
        [-0.0717],
        [-0.0271],
        [-0.1342],
        [-0.1936],
        [-0.1077],
        [-0.0828],
        [-0.1227],
        [-0.1213],
        [-0.0318],
        [-0.1139],
        [-0.0808],
        [-0.0140],
        [-0.0160],
        [-0.0659],
        [ 0.0056],
        [-0.1207],
        [-0.

## Gather Betting Data

In [ ]:
def clean_bet_data(df = betting_data):
    
    # read betting data
    betting_data = pd.read_csv('./data/nba_betting_data_2010_present.csv')
    betting_data = betting_data.dropna()
    
    
    df['date'] = pd.to_datetime(df['date'])
    df['home_team_abbr'] = df['home_team_abbr'].replace({'NY':'NYK',
                                                                            'GS':'GSW',
                                                                            'SA':'SAS',
                                                                            'BK':'BKN',
                                                                            'NO':'NOP',
                                                                            'PHO':'PHX'
                                                                                        }
                                                                             )
    df['away_team_abbr'] = df['away_team_abbr'].replace({'NY':'NYK',
                                                                            'GS':'GSW',
                                                                            'SA':'SAS',
                                                                            'BK':'BKN',
                                                                            'NO':'NOP',
                                                                            'PHO':'PHX'}
                                                                           )

    teams = df['home_team_abbr'].unique()

    df = df.sort_values(['date'])
    df['rest'] = np.nan
    for team in teams:
        team_data = df.loc[(df['home_team_abbr'] == team) | (df['away_team_abbr'] == team)]
        team_data['rest'] = (team_data['date'] - team_data['date'].shift(1)) / np.timedelta64(1, 'D')
        df.loc[(df['home_team_abbr'] == team) | (df['away_team_abbr'] == team), 'rest'] = team_data['rest']


    df['point_diff'] = df['home_score'] - df['away_score']
    df['point_total'] = df['home_score'] + df['away_score']
    df['covered_spread'] = (df['home_score'] + df['spread'] > df['away_score']).astype(int)
    df['over'] = (df['point_total'] > df['total']).astype(int)
    
    df['prev_cover'] = df['covered_spread'].shift(1)
    df['prev2_cover'] = df['covered_spread'].shift(2)
    
    relevant_betting = df[['date', 'home_team_abbr',  'away_team_abbr',
                           'home_score', 'away_score', 'point_diff', 
                           'rest', 'point_total', 'spread', 'total',
                          'covered_spread', 'over', 'prev_cover', 'prev2_cover']]

    return relevant_betting

bet_data_clean = clean_bet_data()